### 필수 라이브러리 설치

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2
!pip3 install -q -U wandb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.5.3 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


### 데이터 다운

In [ ]:
# !wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000302/data/data.tar.gz
# !tar -xvf data.tar.gz
# !rm -rf data.tar.gz

### 허깅페이스 로그인

In [ ]:
!huggingface-cli login --token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /data/ephemeral/home/.cache/huggingface/token
Login successful


### WandB login

In [ ]:
!wandb login

wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


### 라이브러리 임포트

In [ ]:
import torch
from datasets import Dataset, DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
import wandb
import time
from tqdm import tqdm
import warnings

## 모델 불러오기

In [ ]:
# LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
# Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_id = "whybe-choi/OPEN-SOLAR-KO-10.7B-sum"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 데이터 불러오기

In [ ]:
train_df = pd.read_csv('../data/train.csv')
valid_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

In [ ]:
display(train_df.head())
display(valid_df.head())
display(test_df.head())

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? \n#Person2#: 요즘...",#Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대...,의사에게 상담하기
1,dev_1,"#Person1#: 헤이, 지미. 나중에 운동하러 가자.\n#Person2#: 그래...",#Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.,운동하기
2,dev_2,#Person1#: 나는 더 이상 건강에 해로운 음식을 먹는 것을 멈춰야 해.\n#...,"#Person1#은 건강에 해로운 음식을 먹는 것을 멈추려는 계획을 세우고, #Pe...",건강한 음식
3,dev_3,"#Person1#: UFO를 믿으세요?\n#Person2#: 물론이죠, 그들은 저기...",#Person2#는 UFO를 믿고 꿈에서 그들을 볼 수 있다고 말한다. #Perso...,UFO와 외계인
4,dev_4,#Person1#: 오늘 학교에 갔어?\n#Person2#: 당연하지. 너는?\n#...,#Person1#은 오늘 학교에 가지 않았다. #Person2#는 내일 수업을 빼먹...,학교 가기


,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...


In [ ]:
# DataFrame을 Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

# DatasetDict 생성
dataset = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset,
    'test': test_dataset
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['fname', 'dialogue', 'summary', 'topic'],
        num_rows: 12457
    })
    valid: Dataset({
        features: ['fname', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
    test: Dataset({
        features: ['fname', 'dialogue'],
        num_rows: 499
    })
})

In [ ]:
# 데이터 형태 확인
dataset['train'][0]

{'fname': 'train_0',
 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.',
 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.',
 'topic': '건강검진 받기'}

In [ ]:
dataset['train']

Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic'],
    num_rows: 12457
})

## 프롬프트 형태로 변환하기

In [ ]:
def prompt_formatter(sample):
    return f"""<s>### Instruction:
당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다.

### Dialogue:
{sample['dialogue']}

### Summary:
{sample['summary']}</s>"""

In [ ]:
print(prompt_formatter(dataset['train'][0]))

<s>### Instruction:
당신은 대화를 요약해주는 유능한 AI입니다. 당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. 당신의 대답은 오직 제공된 대화에만 근거해야 합니다.

### Dialogue:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

### Summary:
스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.</s>


## 모델 학습하기

### WandB 연동

In [ ]:
wandb.init(
    entity='NLP-team3',
    project='OPEN-SOLAR-KO-10.7B',
    name=f"OPEN-SOLAR-KO-10.7B-{str(int(time.time()))}",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: whybe-choi (NLP-team3). Use `wandb login --relogin` to force relogin


In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=4,
    save_strategy="epoch",
    learning_rate=4e-4,  ### 2e-4
    optim="paged_adamw_32bit",
    bf16=True,
    fp16=False,
    tf32=True,
    max_grad_norm=1.0,
    warmup_ratio=0.06,
    lr_scheduler_type="cosine",
    disable_tqdm=False,
    weight_decay=0.01,
    report_to='wandb',     # Logging에 wandb를 이용함
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['valid'],
    max_seq_length=1024,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_formatter,
    args=args,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
4,1.389100
8,1.291400
12,1.320400
16,1.311700
20,1.304900
24,1.272700
28,1.323600
32,1.235900
36,1.212300
40,1.199600


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=54, training_loss=1.2607893458119146, metrics={'train_runtime': 712.7872, 'train_samples_per_second': 0.61, 'train_steps_per_second': 0.076, 'total_flos': 2.7928224004571136e+16, 'train_loss': 1.2607893458119146, 'epoch': 2.92})

In [ ]:
# wandb 종료
wandb.finish()

train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇██
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇██
train/grad_norm,▃▁▂▁▁▁▃▄▅▆█▇▇
train/learning_rate,███▇▆▆▅▄▃▂▂▁▁
train/loss,█▅▆▆▅▄▆▃▂▂▃▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,2.92


In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_id = "whybe-choi/OPEN-SOLAR-KO-10.7B-sum"
ADAPTER_MODEL = "lora_adapter"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.save_pretrained('OPEN-SOLAR-KO-10.7B-sum-val')
tokenizer.save_pretrained('OPEN-SOLAR-KO-10.7B-sum-val')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('OPEN-SOLAR-KO-10.7B-sum-val/tokenizer_config.json',
 'OPEN-SOLAR-KO-10.7B-sum-val/special_tokens_map.json',
 'OPEN-SOLAR-KO-10.7B-sum-val/tokenizer.json')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
MODEL_SAVE_HUB_PATH = 'whybe-choi/OPEN-SOLAR-KO-10.7B-sum-val' # 여기에 {본인의 허깅페이스 허브}/{저장하고자 하는 이름} 형태로 작성
HUGGINGFACE_AUTH_TOKEN = '' # 허깅페이스 write token

model.push_to_hub(
   MODEL_SAVE_HUB_PATH,
   use_temp_dir=True,
   use_auth_token=HUGGINGFACE_AUTH_TOKEN
)
tokenizer.push_to_hub(
   MODEL_SAVE_HUB_PATH,
   use_temp_dir=True,
   use_auth_token=HUGGINGFACE_AUTH_TOKEN
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/whybe-choi/OPEN-SOLAR-KO-10.7B-sum-val/commit/86c7412e4a64a7a6bb284b04b3d5218a279a0b06', commit_message='Upload tokenizer', commit_description='', oid='86c7412e4a64a7a6bb284b04b3d5218a279a0b06', pr_url=None, pr_revision=None, pr_num=None)

## 추론

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
FINETUNE_MODEL = "whybe-choi/OPEN-SOLAR-KO-10.7B-sum-val"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, low_cpu_mem_usage=True, quantization_config=bnb_config, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(FINETUNE_MODEL)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=192)

In [ ]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['dialogue'])):
        prompt = f"""<s>### Instruction:
당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다.

### Dialogue:
{example['dialogue'][i]}

### Summary:
"""
        output_texts.append(prompt)
    return output_texts

In [ ]:
test_data = dataset['test']
print(generate_prompt(test_data[:1])[0])

<s>### Instruction:
당신은 대화를 요약해주는 유능한 AI입니다. 당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. 당신의 대답은 오직 제공된 대화에만 근거해야 합니다.

### Dialogue:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.

### Summary:



In [ ]:
prompt = generate_prompt(test_data[:1])[0]

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.1,
)
summary = outputs[0]["generated_text"][len(prompt):]
print(summary)

더슨 씨는 #Person1#의 지시로 내부 및 외부 통신 모두에 적용되는 새로운 정책에 대한 메모를 작성하고 있습니다.


In [ ]:
warnings.filterwarnings(action='ignore')

submission = pd.read_csv("../data/sample_submission.csv")
prompts = generate_prompt(dataset['test'])

for idx, prompt in enumerate(tqdm(prompts)):
    outputs = pipe_finetuned(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.1,
    )
    summary = outputs[0]["generated_text"][len(prompt):]

    if idx % 50 == 0:
        print("="*25, "[ 대화 ]", "="*25)
        print(dataset['test'][idx]["dialogue"])
        print("="*25, "[ 요약 ]", "="*25)
        print(summary)
        print()

    submission.loc[idx, 'summary'] = summary.strip()

  0%|          | 1/499 [00:02<19:45,  2.38s/it]

========================= [ 대화 ] =========================
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.
========================= [ 요약 ] =========================
더슨 씨는 #Person1#의 지시로 내부 및 외부 통신 모두에 적용되는 새로운 정책에 대한 메모를 작성하고

 10%|█         | 51/499 [02:15<20:45,  2.78s/it]

========================= [ 대화 ] =========================
#Person1#: 택시!
#Person2#: 어디로 가시겠습니까, 손님?
#Person1#: 프렌드십 호텔이요.
#Person2#: 알겠습니다, 여기서 멀지 않아요.
#Person1#: 중요한 일이 있어서, 속도를 높일 수 있나요?
#Person2#: 네, 최선을 다하겠습니다. 도착했습니다.
#Person1#: 정말 빠르네요! 얼마나 내야 하나요?
#Person2#: 미터기에는 15위안이 나옵니다.
#Person1#: 여기 20위안 있습니다, 거스름돈은 가지세요.
#Person2#: 정말 감사합니다.
========================= [ 요약 ] =========================
#Person1#이 #Person2#의 택시에 타고, #Person2#는 #Person1#을 빠르게 목적지로 데려다 준다.



 20%|██        | 101/499 [04:25<15:49,  2.39s/it]

...

 80%|████████  | 401/499 [17:12<04:27,  2.73s/it]

========================= [ 대화 ] =========================
#Person1#: 안녕하세요. 세입자 지원 센터입니다.
#Person2#: 안녕하세요. 제 집주인과 문제가 있어요. 집주인은 충분히 좋은 사람이지만, 그와 저는 수리 비용에 대해 합의를 이루지 못하고 있어요.
#Person1#: 그가 수리를 하지 않는 것인가요?
#Person2#: 그가 수리를 하지 않는 것은 아니에요. 그냥 너무 오래 걸려요. 처음에 이사 왔을 때, 히터가 고장나서 3개월이나 걸려서 수리했고, 지난달에 제 30번째 생일이었어요. 친구들이 저에게 놀랄만한 파티를 열어줬어요. 많은 음식과 맥주를 가지고 왔고, 심지어 록 앤 롤 밴드까지 있었어요. 밤새도록 파티를 했고.
#Person1#: 그리고 이웃들이 소음에 대해 불평을 했나요.
#Person2#: 아니요, 건물에 있는 모든 사람들이 왔어요. 정말 멋진 파티였어요. 불행히도, 몇몇 사람들이 실수로 거실 창문을 깨뜨렸어요. 다음 날, 저는 수리공을 불러서 창문을 고쳤어요. 그가 이미 여기에 있었기 때문에, 저는 그에게 2개월 이상 집주인에게 불평하고 있던 고장난 세탁기를 고치게 했어요. 그리고 지난 주, 저는 수리 비용을 제외하고 집에 대한 돈을 우편으로 보냈어요. 하지만 오늘 아침, 집주인이 화를 내며 전화해서 그가 수리 비용을 지불하지 않겠다고 했어요. 그게 공정해 보이지 않아요. 제가 어떻게 해야 할까요?
========================= [ 요약 ] =========================
#Person2#는 #Person1#에게 집주인이 수리 비용을 지불하지 않고 있다고 말합니다. #Person2#는 이것이 불공정하다고 생각하고 #Person1#에게 어떻게 해야 할지 묻습니다.



 90%|█████████ | 451/499 [19:29<02:07,  2.65s/it]

========================= [ 대화 ] =========================
#Person1#: 실례합니다.
#Person2#: 네?
#Person1#: 피크 트램까지 어떻게 가는지 알려주실 수 있나요?
#Person2#: 물론입니다. 퀸스 로드를 따라가세요...
#Person1#: 퀸스 로드를 따라...
#Person2#: 네, 그리고 힐튼 호텔에서 오른쪽으로 꺾으세요.
#Person1#: 힐튼에서 오른쪽으로.
#Person2#: 그런 다음 가든 로드를 따라 올라가서 대성당을 지나가세요.
#Person1#: 대성당이요?
#Person2#: 네. 그 다음 신호등에서 길을 건너세요. 피크 트램은 바로 앞에 있습니다. 찾을 수 있을 거예요.
#Person1#: 정말 친절하시네요. 감사합니다. 어... 연필 가지고 계신가요?
#Person2#: 네. 왜요?
#Person1#: 그 방법을 다시 한 번 말해주실 수 있나요? 적어두는 게 좋을 것 같아요.
========================= [ 요약 ] =========================
#Person2#는 #Person1#에게 피크 트램으로 가는 길을 알려줍니다. #Person1#은 #Person2#의 도움을 받아 메모할 예정입니다.



100%|██████████| 499/499 [21:31<00:00,  2.59s/it]


In [ ]:
submission.to_csv("../submission_solar.csv", index=False)